## GPU-accelerated approximate k-mer counting using a bloom filter
#### Integrating Project | G01 
##### Members: Bawa, Lejano, Roxas

### C serialized version

For the serialized version implemented in C, we utilized two existing libraries namely libbloom and uthash. We also created 2 helper files to load the FASTA sequence and provide a ground truth for correctness. Phase 1 and Phase 2 implementations were then implemented as functions alongside the necessary structs and the main pipeline.

#### Bloom filter setup

In [1]:
!git clone https://github.com/jvirkki/libbloom.git

fatal: destination path 'libbloom' already exists and is not an empty directory.


In [2]:
# build libbloom
!cd libbloom && make

make: Nothing to be done for 'all'.


In [3]:
# verify build
!ls libbloom


bloom.c  build	    docs     Makefile  murmur2
bloom.h  ChangeLog  LICENSE  misc      README


#### Hash table setup

In [4]:
!wget -q https://raw.githubusercontent.com/troydhanson/uthash/master/src/uthash.h -O uthash.h

In [5]:
!gcc test_bloom.c -Ilibbloom libbloom/build/libbloom.a -lm -o test_bloom.out


#### Build helper files

In [6]:
%%writefile fasta_reader.h
#ifndef FASTA_READER_H
#define FASTA_READER_H

#ifdef __cplusplus
extern "C" {
#endif

// loads a DNA sequence from a FASTA file
// returns a pointer to a null-terminated string containing the sequence
char* load_sequence(const char* filename);

#ifdef __cplusplus
}
#endif

#endif // FASTA_READER_H

Overwriting fasta_reader.h


In [7]:
%%writefile fasta_reader.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <ctype.h>
#include "fasta_reader.h"

char* load_sequence(const char* filename) {
    // open the FASTA file, return NULL on failure
    FILE* fp = fopen(filename, "r");
    if (!fp) {
        perror("Failed to open file");
        return NULL;
    }

    // initialize variables for dynamic sequence storage
    size_t capacity = 1024;
    size_t length = 0;
    char* sequence = malloc(capacity);

    char line[1024];
    while (fgets(line, sizeof(line), fp)) {
        if (line[0] == '>') continue; // skip header lines
        
        for (int i = 0; line[i]; i++) {
            char c = toupper(line[i]);
            if (c == 'A' || c == 'T' || c == 'G' || c == 'C' || c == 'U') {
                // resize if necessary
                if (length + 1 >= capacity) {
                    capacity *= 2;
                    sequence = realloc(sequence, capacity);
                }

                sequence[length++] = c; // append nucleotide before incrementing length
            }
        }
    }

    fclose(fp);
    sequence[length] = '\0'; // terminate the string
    return sequence;
}


Overwriting fasta_reader.c


In [8]:
%%writefile ground_truth.h
#ifndef GROUND_TRUTH_H
#define GROUND_TRUTH_H

#include <stddef.h>
#include "uthash.h"

#ifdef __cplusplus
extern "C" {
#endif

// structure for k-mer entries in the hash table
typedef struct KmerEntry {
    char kmer[64]; // assuming k <= 63
    int count;
    UT_hash_handle hh;
} KmerEntry;

// computes the ground truth k-mer counts from the given sequence
// parameters: sequence: input DNA sequence, k: k-mer size, length: length of the sequence, total_unique_kmers: pointer to output parameter for total unique k-mers, 
// true_non_singletons: pointer to output parameter for non-singleton k-mers, truth_table: output hash table of k-mer counts
void get_ground_truth(const char *sequence, int k, size_t length, size_t *total_unique_kmers, size_t *true_non_singletons, KmerEntry **truth_table);

// frees the memory allocated for the truth table
void free_truth_table(KmerEntry *truth_table);

#ifdef __cplusplus
}
#endif

#endif // GROUND_TRUTH_H

Overwriting ground_truth.h


In [9]:
%%writefile ground_truth.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "fasta_reader.h"
#include "ground_truth.h"

void get_ground_truth(const char *sequence, int k, size_t length, size_t *total_unique_kmers, size_t *true_non_singletons, KmerEntry **truth_table) {
    // initialize output parameters
    *truth_table = NULL;
    *total_unique_kmers = 0;
    *true_non_singletons = 0;
    
    if (length < (size_t)k) {
        printf("Sequence shorter than k-mer length.\n");
        return;
    }

    // iterate over the sequence
    for (size_t i = 0; i <= length - (size_t)k; i++) {

        // extract k-mer
        char kmer[k + 1];
        strncpy(kmer, sequence + i, k);
        kmer[k] = '\0';

        // update hash table
        KmerEntry *entry;
        HASH_FIND_STR(*truth_table, kmer, entry);

        if (entry) {
            entry->count++;
        } else {
            entry = malloc(sizeof(KmerEntry));
            strcpy(entry->kmer, kmer);
            entry->count = 1;
            HASH_ADD_STR(*truth_table, kmer, entry);
        }
    }

    // compute total unique k-mers and true non-singletons, keep only true non-singletons in the table
    KmerEntry *curr, *tmp;
    HASH_ITER(hh, *truth_table, curr, tmp) {
        (*total_unique_kmers)++;
        if (curr->count < 2) {
            HASH_DEL(*truth_table, curr);
            free(curr);
        } else {
            (*true_non_singletons)++;
        }
    }

    printf("Sequence length: %zu\n", length);
    printf("Total unique k-mers: %zu\n", *total_unique_kmers);
    printf("True non-singletons: %zu\n", *true_non_singletons);
    printf("\n");
}

void free_truth_table(KmerEntry *truth_table) {
    KmerEntry *curr, *tmp;
    HASH_ITER(hh, truth_table, curr, tmp) {
        HASH_DEL(truth_table, curr);
        free(curr);
    }
}


Overwriting ground_truth.c


#### Main Pipeline

In [10]:
%%writefile serialized.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <string.h>
#include "libbloom/bloom.h"
#include "fasta_reader.h"
#include "ground_truth.h"

// Data structures

// structure for candidate list
typedef struct {
    char **items;
    size_t size;
    size_t capacity;
} CandidateList;

// structure for seen k-mers hash set
typedef struct {
    char kmer[64];
    UT_hash_handle hh;
} KmerSeen;

// structure for k-mer counts in phase 2
typedef struct {
    char kmer[64];
    int count;
    UT_hash_handle hh;
} KmerCount;

// CandidateList helpers

// initialize candidate list
void init_candidate_list(CandidateList *list) {
    list->size = 0; // how many we have
    list->capacity = 1024; // how many we can store
    list->items = malloc(list->capacity * sizeof(char*));
}

// add candidate k-mer to the list
void add_candidate(CandidateList *list, const char *kmer) {
    if (list->size >= list->capacity) {
        list->capacity *= 2;
        list->items = realloc(list->items, list->capacity * sizeof(char*));
    }
    list->items[list->size++] = strdup(kmer);
}

// free candidate list
void free_candidate_list(CandidateList *list) {
    for (size_t i = 0; i < list->size; i++) free(list->items[i]);
    free(list->items);
}

// Phase 1

void phase1_filter(const char *sequence, int k, size_t entries, double error_rate, CandidateList *out_candidates) {

    // initialize bloom filter of libbloom
    struct bloom filter;
    bloom_init(&filter, entries, error_rate);

    // initialize candidate list, length of sequence, and seen hash set
    init_candidate_list(out_candidates);
    size_t length = strlen(sequence);
    KmerSeen *seen = NULL;

    for (size_t i = 0; i <= length - (size_t)k; i++) {
        char kmer[k + 1];
        strncpy(kmer, sequence + i, k);
        kmer[k] = '\0';

        if (bloom_check(&filter, kmer, k)) {
            // check if already a candidate
            KmerSeen *tmp;
            HASH_FIND_STR(seen, kmer, tmp);
            if (tmp == NULL) {
                add_candidate(out_candidates, kmer);

                // mark as seen
                tmp = malloc(sizeof(KmerSeen));
                strcpy(tmp->kmer, kmer);
                HASH_ADD_STR(seen, kmer, tmp);
            }
        }

        // add k-mer to bloom filter
        bloom_add(&filter, kmer, k);
    }

    // free hash set
    KmerSeen *curr, *tmp;
    HASH_ITER(hh, seen, curr, tmp) {
        HASH_DEL(seen, curr);
        free(curr);
    }

    // free bloom filter
    bloom_free(&filter);
}

// Phase 2

void phase2_count(const char *sequence, CandidateList *candidates, int k) {
    KmerCount *entries = NULL;
    KmerSeen *candidate_hash = NULL;
    size_t length = strlen(sequence);

    // build hash set for candidates for quick lookup
    for (size_t i = 0; i < candidates->size; i++) {
        KmerSeen *tmp = malloc(sizeof(KmerSeen));
        strcpy(tmp->kmer, candidates->items[i]);
        HASH_ADD_STR(candidate_hash, kmer, tmp);
    }

    for (size_t i = 0; i <= length - (size_t)k; i++) {
        char kmer[k + 1];
        strncpy(kmer, sequence + i, k);
        kmer[k] = '\0';

        // check if k-mer is in candidates
        KmerSeen *found = NULL;
        HASH_FIND_STR(candidate_hash, kmer, found);
        if (!found) continue;

        // add to entries if candidate
        KmerCount *entry;
        HASH_FIND_STR(entries, kmer, entry);
        if (entry) entry->count++;
        else {
            entry = malloc(sizeof(KmerCount));
            strcpy(entry->kmer, kmer);
            entry->count = 1;
            HASH_ADD_STR(entries, kmer, entry);
        }
    }

    // free entries hash table
    KmerCount *curr, *tmp;
    HASH_ITER(hh, entries, curr, tmp) {
        HASH_DEL(entries, curr);
        free(curr);
    }

    // free candidate hash set
    KmerSeen *c_curr, *c_tmp;
    HASH_ITER(hh, candidate_hash, c_curr, c_tmp) {
        HASH_DEL(candidate_hash, c_curr);
        free(c_curr);
    }
}

// main
int main() {
    // read sequence from FASTA file
    char* sequence = load_sequence("Homo_Sapiens_Ch17.fasta");
    if (!sequence) return 1;

    // initialize parameters
    int k = 21;
    double error_rate = 0.01;
    size_t length = strlen(sequence);

    // initialize varying bloom filter sizes 
    // where: larger entries → more memory → fewer false positives -> less phase 2 work 
    // smaller entries → smaller memory → more false positives -> tons of phase 2 work.
    // bloom size close to real number of unique k-mers → optimal.
    size_t filter_sizes[7];
    filter_sizes[0] = 10000;
    filter_sizes[1] = 50000;
    filter_sizes[2] = 100000;
    filter_sizes[3] = 2500000;
    filter_sizes[4] = 5000000;
    filter_sizes[5] = 10000000;
    filter_sizes[6] = length - (size_t)k + 1; // estimated unique kmers
    int num_sizes = sizeof(filter_sizes) / sizeof(filter_sizes[0]);

    // timer variables
    clock_t p1_start, p2_start, p1_end, p2_end;
    double p1_time_taken, p2_time_taken, total_time_taken;

    // get ground-truth
    KmerEntry *truth_table = NULL;
    size_t total_unique_kmers = 0, true_non_singletons = 0;
    get_ground_truth(sequence, k, length, &total_unique_kmers, &true_non_singletons, &truth_table);

    printf("Note: false positive rate = (candidates - true non-singletons) / true non-singletons\n");
    printf("The following results are averaged over 10 runs per bloom filter size.\n");
    printf("%-12s %-12s %-12s %-14s %-12s %-15s %-18s %-16s\n",
        "BloomSize", "Phase1_ms", "Phase2_ms", "TotalTime_ms",
        "Candidates", "FalsePositives", "FalsePositiveRate", "FalseNegatives");

    for (int i = 0; i<num_sizes; i++) {
        // get current filter size and initialize timing sums
        size_t entries = filter_sizes[i];
        double sum_p1 = 0.0, sum_p2 = 0.0, sum_total = 0.0;

        // initialize CandidateList
        CandidateList candidates;

        for (int run=0; run<10; run++) {

            // call phase 1
            p1_start = clock();
            phase1_filter(sequence, k, entries, error_rate, &candidates);
            p1_end = clock();

            // call phase 2
            p2_start = clock();
            phase2_count(sequence, &candidates, k);
            p2_end = clock();

            // compute times
            p1_time_taken = ((double)(p1_end-p1_start))*1E3/CLOCKS_PER_SEC;
            p2_time_taken = ((double)(p2_end-p2_start))*1E3/CLOCKS_PER_SEC;
            total_time_taken = p1_time_taken + p2_time_taken;

            // add times to sums
            sum_p1 += p1_time_taken;
            sum_p2 += p2_time_taken;
            sum_total += total_time_taken;        

            // free candidates for next run unless last run
            if (run < 9) {free_candidate_list(&candidates);
        }
        }

        // compute for false positives, false positive rate, and false negatives

        KmerSeen *candidate_hash = NULL;
        for (size_t i = 0; i < candidates.size; i++) {
            KmerSeen *tmp = (KmerSeen*)malloc(sizeof(KmerSeen));
            strcpy(tmp->kmer, candidates.items[i]);
            HASH_ADD_STR(candidate_hash, kmer, tmp);
        }
        
        size_t false_positives = 0;
        size_t false_negatives = 0;
        
        KmerEntry *curr, *tmp;
        HASH_ITER(hh, truth_table, curr, tmp) {
            // Check if truth kmer is missing in candidates, which means false negative
            KmerSeen *found_in_candidates = NULL;
            HASH_FIND_STR(candidate_hash, curr->kmer, found_in_candidates);
            if (!found_in_candidates) false_negatives++;
        }

        for (size_t i = 0; i < candidates.size; i++) {
            KmerEntry *found_in_truth = NULL;
            HASH_FIND_STR(truth_table, candidates.items[i], found_in_truth);
            if (!found_in_truth) false_positives++;
        }

        double fp_rate = true_non_singletons > 0 ?  (double)false_positives / (double)true_non_singletons : 0.0;
        
        printf("%-12zu %-12.2f %-12.2f %-14.2f %-12zu %-15zu %-18.4f %-16zu\n", entries, sum_p1 / 10.0, sum_p2 / 10.0, sum_total / 10.0, candidates.size, false_positives, fp_rate, false_negatives);

        // free candidate list
        free_candidate_list(&candidates);
    }
    
    // free ground truth table and sequence
    free_truth_table(truth_table);
    free(sequence);
    return 0;
}

Overwriting serialized.c


In [11]:
!gcc serialized.c fasta_reader.c ground_truth.c -Ilibbloom libbloom/build/libbloom.a -lm -o serialized.out
!./serialized.out

Sequence length: 81188
Total unique k-mers: 76575
True non-singletons: 2199

Note: false positive rate = (candidates - true non-singletons) / true non-singletons
The following results are averaged over 10 runs per bloom filter size.
BloomSize    Phase1_ms    Phase2_ms    TotalTime_ms   Candidates   FalsePositives  FalsePositiveRate  FalseNegatives  
10000        193.15       480.73       673.88         43148        40949           18.6216            0               
50000        42.81        78.77        121.58         3129         930             0.4229             0               
100000       35.81        37.91        73.72          2244         45              0.0205             0               
2500000      182.21       34.19        216.40         2199         0               0.0000             0               
5000000      184.27       34.32        218.60         2199         0               0.0000             0               
10000000     217.77       34.50        252.27        

### CUDA Version

#### CUDA Setup

In [12]:
import os
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

In [13]:
%%writefile parallel.cu
#include <stdio.h>
#include <stdint.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <cuda_runtime.h>
#include "fasta_reader.h"
#include "ground_truth.h"

// Data structures

// structure for candidate list
typedef struct {
    char **items;
    size_t size;
    size_t capacity;
} CandidateList;

// structure for seen k-mers hash set
typedef struct {
    char kmer[64];
    UT_hash_handle hh;
} KmerSeen;

// structure for k-mer counts in phase 2
typedef struct {
    char kmer[64];
    int count;
    UT_hash_handle hh;
} KmerCount;

// Bloom filter: Implemented as bit array with atomic operations instead of using libbloom

// Hash functions
// DJB2
__device__ uint32_t hash1(const char* kmer, int k) {
    uint32_t hash = 5381;
    for (int i = 0; i < k; i++) {
        hash = ((hash << 5) + hash) + (uint32_t)kmer[i];
    }
    return hash;
} // from http://www.cse.yorku.ca/~oz/hash.html

// SDBM
__device__ uint32_t hash2(const char* kmer, int k) {
    uint32_t hash = 0;
    for (int i = 0; i < k; i++) {
        hash = (uint32_t)kmer[i] + (hash << 6) + (hash << 16) - hash;
    }
    return hash;
} // from http://www.cse.yorku.ca/~oz/hash.html

// FNV-1a
__device__ uint32_t hash3(const char* kmer, int k) {
    uint32_t hash = 2166136261u;
    for (int i = 0; i < k; i++) {
        hash ^= (uint32_t)kmer[i];
        hash *= 16777619u;
    }
    return hash;
} // from: https://ssojet.com/hashing/fnv-1a-in-c/

// CUDA Phase 1 Kernel

__global__ void phase1_filter_kernel(const char* sequence, size_t seq_length, int k, uint32_t* bloom_filter, size_t bloom_size_bits, char* candidate_buffer, int* candidate_flags, size_t max_kmers) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    
    // Grid-stride loop over all possible k-mer positions
    for (size_t i = index; i <= seq_length - k; i += stride) {
        char kmer[64];
        for (int j = 0; j < k; j++) {
            kmer[j] = sequence[i + j];
        }
        kmer[k] = '\0';
        
        // Compute hash positions
        uint32_t h1 = hash1(kmer, k) % bloom_size_bits;
        uint32_t h2 = hash2(kmer, k) % bloom_size_bits;
        uint32_t h3 = hash3(kmer, k) % bloom_size_bits;
        
        // Calculate bit positions
        // word = find out which 32-bit word contains the bit
        // bit = find which bit inside that word
        uint32_t word_idx1 = h1 / 32;
        uint32_t bit_idx1 = h1 % 32;
        uint32_t word_idx2 = h2 / 32;
        uint32_t bit_idx2 = h2 % 32;
        uint32_t word_idx3 = h3 / 32;
        uint32_t bit_idx3 = h3 % 32;
        
        // check if all bits are set (query)
        // load the 'word' containing the bit, shift it to the desired bit, and mask by 1
        uint32_t bit1 = (bloom_filter[word_idx1] >> bit_idx1) & 1;
        uint32_t bit2 = (bloom_filter[word_idx2] >> bit_idx2) & 1;
        uint32_t bit3 = (bloom_filter[word_idx3] >> bit_idx3) & 1;
        
        if (bit1 && bit2 && bit3) {
            // Mark as candidate
            candidate_flags[i] = 1;
            
            // Copy k-mer to candidate buffer
            for (int j = 0; j < k; j++) {
                candidate_buffer[i * 64 + j] = kmer[j];
            }
            candidate_buffer[i * 64 + k] = '\0';
        }
        
        // set all bits in bloom filter using atomicOr (insert)
        atomicOr(&bloom_filter[word_idx1], 1u << bit_idx1);
        atomicOr(&bloom_filter[word_idx2], 1u << bit_idx2);
        atomicOr(&bloom_filter[word_idx3], 1u << bit_idx3);
    }
}

// CandidateList helpers 

// initialize candidate list
void init_candidate_list(CandidateList *list) {
    list->size = 0;
    list->capacity = 1024;
    list->items = (char**)malloc(list->capacity * sizeof(char*));
}

// add candidate k-mer to the list
void add_candidate(CandidateList *list, const char *kmer) {
    if (list->size >= list->capacity) {
        list->capacity *= 2;
        list->items = (char**)realloc(list->items, list->capacity * sizeof(char*));
    }
    list->items[list->size++] = strdup(kmer);
}

// free candidate list
void free_candidate_list(CandidateList *list) {
    for (size_t i = 0; i < list->size; i++) free(list->items[i]);
    free(list->items);
}

// Phase 1 Wrapper

void phase1_filter_cuda(const char *sequence, int k, size_t entries, double error_rate, CandidateList *out_candidates) {
    size_t seq_length = strlen(sequence);
    size_t num_kmers = seq_length - k + 1;
    
    // calculate bloom filter size in bits
    // m = -(n * ln(p)) / (ln(2)^2) where n=entries, p=error_rate. Source: https://en.wikipedia.org/wiki/Bloom_filter#Approximating_the_number_of_items_in_a_Bloom_filter
    size_t bloom_size_bits = (size_t)(-(double)entries * log(error_rate) / (log(2) * log(2)));
    size_t bloom_size_words = (bloom_size_bits + 31) / 32;
    size_t bloom_bytes = bloom_size_words * sizeof(uint32_t);
    
    // allocate memory on device
    char *d_sequence;
    uint32_t *d_bloom_filter;
    char *d_candidate_buffer;
    int *d_candidate_flags;
    
    cudaMallocManaged(&d_sequence, seq_length + 1);
    cudaMallocManaged(&d_bloom_filter, bloom_bytes);
    cudaMallocManaged(&d_candidate_buffer, num_kmers * 64);
    cudaMallocManaged(&d_candidate_flags, num_kmers * sizeof(int));
    
    // get device ID
    int device = -1;
    cudaGetDevice(&device);
    
    // mem advise
    cudaMemAdvise(d_sequence, seq_length + 1, cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_sequence, seq_length + 1, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    
    // prefetch for page creation
    cudaMemPrefetchAsync(d_sequence, seq_length + 1, cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(d_bloom_filter, bloom_bytes, device, NULL);
    cudaMemPrefetchAsync(d_candidate_buffer, num_kmers * 64, device, NULL);
    cudaMemPrefetchAsync(d_candidate_flags, num_kmers * sizeof(int), device, NULL);
    
    // initialize data
    strcpy(d_sequence, sequence);
    memset(d_bloom_filter, 0, bloom_bytes);
    memset(d_candidate_flags, 0, num_kmers * sizeof(int));
    
    // prefetch from CPU to GPU
    cudaMemPrefetchAsync(d_sequence, seq_length + 1, device, NULL);
    
    // launch kernel with grid-stride
    size_t n_threads = 1024;
    size_t n_blocks = (num_kmers + n_threads - 1) / n_threads;
    
    // cap blocks for efficiency
    if (n_blocks > 2048) n_blocks = 2048;
    
    phase1_filter_kernel<<<n_blocks, n_threads>>>(d_sequence, seq_length, k, d_bloom_filter, bloom_size_bits, d_candidate_buffer, d_candidate_flags, num_kmers);
    
    // barrier
    cudaDeviceSynchronize();
    
    // prefetch results back to CPU
    cudaMemPrefetchAsync(d_candidate_buffer, num_kmers * 64, cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(d_candidate_flags, num_kmers * sizeof(int), cudaCpuDeviceId, NULL);
    cudaDeviceSynchronize();
    
    // collect candidates (remove duplicates using hash set)
    init_candidate_list(out_candidates);
    KmerSeen *seen = NULL;
    
    for (size_t i = 0; i < num_kmers; i++) {
        if (d_candidate_flags[i]) {
            char *kmer = &d_candidate_buffer[i * 64];
            
            // Check if already added
            KmerSeen *tmp;
            HASH_FIND_STR(seen, kmer, tmp);
            if (tmp == NULL) {
                add_candidate(out_candidates, kmer);
                
                // Mark as seen
                tmp = (KmerSeen*)malloc(sizeof(KmerSeen));
                strcpy(tmp->kmer, kmer);
                HASH_ADD_STR(seen, kmer, tmp);
            }
        }
    }
    
    // Free seen hash set
    KmerSeen *curr, *tmp;
    HASH_ITER(hh, seen, curr, tmp) {
        HASH_DEL(seen, curr);
        free(curr);
    }
    
    // Free device memory
    cudaFree(d_sequence);
    cudaFree(d_bloom_filter);
    cudaFree(d_candidate_buffer);
    cudaFree(d_candidate_flags);
}

// Phase 2

void phase2_count(const char *sequence, CandidateList *candidates, int k, KmerCount **out_counts) {
    KmerSeen *candidate_hash = NULL;
    size_t length = strlen(sequence);

    // build hash set for candidates for quick lookup
    for (size_t i = 0; i < candidates->size; i++) {
        KmerSeen *tmp = (KmerSeen*)malloc(sizeof(KmerSeen));
        strcpy(tmp->kmer, candidates->items[i]);
        HASH_ADD_STR(candidate_hash, kmer, tmp);
    }

    for (size_t i = 0; i <= length - (size_t)k; i++) {
        char kmer[64];
        strncpy(kmer, sequence + i, k);
        kmer[k] = '\0';

        // check if k-mer is in candidates
        KmerSeen *found = NULL;
        HASH_FIND_STR(candidate_hash, kmer, found);
        if (!found) continue;

        // add to entries if candidate
        KmerCount *entry;
        HASH_FIND_STR(*out_counts, kmer, entry);
        if (entry) {
            entry->count++;
        } else {
            entry = (KmerCount*)malloc(sizeof(KmerCount));
            strcpy(entry->kmer, kmer);
            entry->count = 1;
            HASH_ADD_STR(*out_counts, kmer, entry);
        }
    }

    // free candidate hash set
    KmerSeen *c_curr, *c_tmp;
    HASH_ITER(hh, candidate_hash, c_curr, c_tmp) {
        HASH_DEL(candidate_hash, c_curr);
        free(c_curr);
    }
}

// main
int main() {
    // read sequence from FASTA file
    char* sequence = load_sequence("Homo_Sapiens_Ch17.fasta");
    if (!sequence) return 1;

    // initialize parameters
    int k = 12;
    double error_rate = 0.01;
    size_t length = strlen(sequence);

    // initialize varying bloom filter sizes
    size_t filter_sizes[7];
    filter_sizes[0] = 10000;
    filter_sizes[1] = 50000;
    filter_sizes[2] = 100000;
    filter_sizes[3] = 2500000;
    filter_sizes[4] = 5000000;
    filter_sizes[5] = 10000000;
    filter_sizes[6] = length - (size_t)k + 1;
    int num_sizes = sizeof(filter_sizes) / sizeof(filter_sizes[0]);

    // timer variables
    cudaEvent_t p1_start, p1_stop;
    clock_t p2_start, p2_end;
    float p1_time_taken;
    double p2_time_taken;
    
    cudaEventCreate(&p1_start);
    cudaEventCreate(&p1_stop);

    // get ground-truth
    KmerEntry *truth_table = NULL;
    size_t total_unique_kmers = 0, true_non_singletons = 0;
    get_ground_truth(sequence, k, length, &total_unique_kmers, &true_non_singletons, &truth_table);

    printf("Note: false positive rate = (candidates - true non-singletons) / true non-singletons\n");
    printf("The following results are averaged over 10 runs per bloom filter size.\n");
    printf("%-12s %-12s %-12s %-14s %-12s %-15s %-18s %-16s %-20s %-28s\n",
        "BloomSize", "Phase1_ms", "Phase2_ms", "TotalTime_ms",
        "Candidates", "FalsePositives", "FalsePositiveRate", "FalseNegatives", "Average Occur of FN", "Average Occur of TP");

    for (int i = 0; i < num_sizes; i++) {
        size_t entries = filter_sizes[i];
        double sum_p1 = 0.0, sum_p2 = 0.0, sum_total = 0.0;
    
        // accumulators for averaged metrics
        size_t sum_candidates = 0;
        size_t sum_false_positives = 0;
        size_t sum_false_negatives = 0;
        double sum_avg_fn = 0.0;
        double sum_avg_tp = 0.0;

        for (int run = 0; run < 10; run++) {
            CandidateList candidates;
            KmerCount *counts = NULL;
            
            // Phase 1
            cudaEventRecord(p1_start);
            phase1_filter_cuda(sequence, k, entries, error_rate, &candidates);
            cudaEventRecord(p1_stop);
            cudaEventSynchronize(p1_stop);
            cudaEventElapsedTime(&p1_time_taken, p1_start, p1_stop);

            // Phase 2      
            p2_start = clock();
            phase2_count(sequence, &candidates, k, &counts);
            p2_end = clock();
            p2_time_taken = ((double)(p2_end-p2_start))*1E3/CLOCKS_PER_SEC;

            sum_p1 += p1_time_taken;
            sum_p2 += p2_time_taken;
            sum_total += (p1_time_taken + p2_time_taken);

            // Calculate false negatives, false positives, true positives, and average occurrences
            size_t false_positives = 0;
            size_t false_negatives = 0;
            size_t total_fn = 0;
            size_t total_tp = 0;
            size_t true_positives = 0;

            KmerEntry *curr_truth, *tmp_truth;
            HASH_ITER(hh, truth_table, curr_truth, tmp_truth) {
                KmerCount *found_in_counts = NULL;
                HASH_FIND_STR(counts, curr_truth->kmer, found_in_counts);
                if (!found_in_counts) {
                    false_negatives++;
                    total_fn += curr_truth->count;
                }
            } 
            double avg_fn = false_negatives > 0 ? (double)total_fn / (double)false_negatives : 0.0;

            // Count false positives and true positives
            KmerCount *curr_count, *tmp_count;
            HASH_ITER(hh, counts, curr_count, tmp_count) {
                KmerEntry *found_in_truth = NULL;
                HASH_FIND_STR(truth_table, curr_count->kmer, found_in_truth);
                if (!found_in_truth) {
                    false_positives++;
                } else {
                    true_positives++;
                    total_tp += curr_count->count;
                }
            }
            double avg_tp = true_positives > 0 ? (double)total_tp / (double)true_positives : 0.0;
            
            // add metrics
            sum_candidates += candidates.size;
            sum_false_positives += false_positives;
            sum_false_negatives += false_negatives;
            sum_avg_fn += avg_fn;
            sum_avg_tp += avg_tp;

            // Clean up for this run
            free_candidate_list(&candidates);
            
            KmerCount *count_curr, *count_tmp;
            HASH_ITER(hh, counts, count_curr, count_tmp) {
                HASH_DEL(counts, count_curr);
                free(count_curr);
            }
        }

        // calculate averages
        double avg_sum_p1 = sum_p1 / 10.0;
        double avg_sum_p2 = sum_p2 / 10.0;
        double avg_sum_total = sum_total / 10.0;
        double avg_candidates = sum_candidates / 10.0;
        double avg_false_positives = sum_false_positives / 10.0;
        double avg_false_negatives = sum_false_negatives / 10.0;
        double avg_fp_rate = true_non_singletons > 0 ? avg_false_positives / (double)true_non_singletons : 0.0;
        double avg_of_avg_fn = sum_avg_fn / 10.0;
        double avg_of_avg_tp = sum_avg_tp / 10.0;
        
        printf("%-12zu %-12.2f %-12.2f %-14.2f %-12.2f %-15.2f %-18.4f %-16.2f %-20.2f %-28.2f\n", entries, avg_sum_p1, avg_sum_p2, avg_sum_total, avg_candidates, avg_false_positives, avg_fp_rate, avg_false_negatives, avg_of_avg_fn, avg_of_avg_tp);
    }
    
    // Cleanup
    cudaEventDestroy(p1_start);
    cudaEventDestroy(p1_stop);
    free_truth_table(truth_table);
    free(sequence);
    return 0;
}

Overwriting parallel.cu


In [14]:
%%bash
nvcc parallel.cu fasta_reader.c ground_truth.c -o parallel -Wno-deprecated-gpu-targets

In [15]:
%%bash
nvprof ./parallel

==449019== NVPROF is profiling process 449019, command: ./parallel


Sequence length: 81188
Total unique k-mers: 66246
True non-singletons: 4380

Note: false positive rate = (candidates - true non-singletons) / true non-singletons
The following results are averaged over 10 runs per bloom filter size.
BloomSize    Phase1_ms    Phase2_ms    TotalTime_ms   Candidates   FalsePositives  FalsePositiveRate  FalseNegatives   Average Occur of FN  Average Occur of TP         
10000        27.55        52.75        80.31          2225.60      930.30          0.2124             3084.70          2.83                 8.22                        
50000        22.52        31.41        53.93          1267.70      29.80           0.0068             3142.10          2.85                 8.37                        
100000       19.85        29.97        49.82          1370.70      3.80            0.0009             3013.10          2.77                 8.01                        
2500000      33.84        36.28        70.11          1678.40      0.00            0.0000  

==449019== Profiling application: ./parallel
==449019== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  192.60ms        70  2.7514ms  764.09us  5.7195ms  phase1_filter_kernel(char const *, unsigned long, int, unsigned int*, unsigned long, char*, int*, unsigned long)
      API calls:   53.24%  1.15040s         2  575.20ms  24.436us  1.15038s  cudaEventCreate
                   24.84%  536.87ms       490  1.0957ms  47.809us  6.2284ms  cudaMemPrefetchAsync
                    9.10%  196.54ms       140  1.4039ms  21.478us  5.7573ms  cudaDeviceSynchronize
                    7.38%  159.51ms       280  569.68us  27.432us  1.7198ms  cudaFree
                    3.23%  69.895ms       280  249.63us  8.4880us  22.040ms  cudaMallocManaged
                    0.94%  20.317ms        70  290.25us  104.82us  2.3151ms  cudaLaunchKernel
                    0.82%  17.693ms       140  126.38us  48.811us  295.66us  cudaEventRec